In [8]:
import pandas as pd

from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import SelectKBest,f_classif

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

def create_base_train_data(x,y):
    selector=SelectKBest(score_func=f_classif,k=5).fit(x,y)
    x_select=selector.transform(x)
    select_features=selector.get_support(1)
    new_x,new_y=SMOTE(sampling_strategy=0.2,random_state=SEED).fit_resample(x_select,y)
    new_x=PolynomialFeatures(degree=2).fit_transform(new_x)
    return new_x,new_y,select_features

def create_base_test_data(x,feature):
    new_x=x[x.columns[feature]]
    new_x=PolynomialFeatures(degree=2).fit_transform(new_x)
    return new_x

def predict_score(model,x_train,y_train,x_test,y_test,name):
    model.fit(x_train,y_train)
    y_predict=model.predict(x_test)
    f1[name]=f1_score(y_test,y_predict)
    auc_score[name]=roc_auc_score(y_test,y_predict)

SEED=42
df=pd.read_csv("dataset/rb.csv")
x=df.drop(["class","smell"],axis=1)
y=df[["smell"]]
f1={}
auc_score={}

In [9]:
print("x:",x.columns,"\n")
print("y:",y.columns,"\n")
rate=y.value_counts()[1]/y.value_counts()[0]
print(y.value_counts())
print("割合:",rate)

x: Index(['dit', 'fanin', 'fanout', 'lcc', 'lcom*', 'loc', 'noc', 'rfc',
       'totalFieldsQty', 'innerClassesQty', 'totalMethodsQty', 'wmc'],
      dtype='object') 

y: Index(['smell'], dtype='object') 

smell
0.0      32409
1.0        176
Name: count, dtype: int64
割合: 0.005430590268135395


In [10]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=SEED)
x_train_base,y_train_base,features=create_base_train_data(x_train,y_train)
x_test_base=create_base_test_data(x_test,features)

c:\Users\sugii syuji\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [11]:
name="DT"
DT_prameter={
    "criterion":["gini","entropy"],
    "max_depth":list(range(1,11)),
    "min_samples_split":list(range(2,25)),
    "min_samples_leaf":list(range(1,20)),
    "max_features":["sqrt","log2",None]
    }
DT=DecisionTreeClassifier(random_state=SEED)

DT_model=RandomizedSearchCV(DT, param_distributions=DT_prameter,cv=StratifiedKFold(n_splits=10),random_state=SEED)
predict_score(DT_model,x_train_base,y_train_base,x_test_base,y_test,name)


In [12]:
name="RF"
RF_prameter={    
    'n_estimators': [100, 200, 300, 400, 500],
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 5, 10, 15, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
    }
RF=RandomForestClassifier(random_state=SEED)

RF_model=RandomizedSearchCV(RF, param_distributions=RF_prameter,cv=StratifiedKFold(n_splits=10),random_state=SEED)
predict_score(RF_model,x_train_base,y_train_base,x_test_base,y_test,name)


c:\Users\sugii syuji\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\sugii syuji\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\sugii syuji\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\sugii syuji\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\b

In [13]:
name="GBR"
GBR_prameter={
    'n_estimators': [10, 100, 1000],
    'learning_rate': [0.001, 0.01, 0.1],
    'subsample': [0.5, 0.7, 1.0],
    'max_depth': [3, 7, 9]
  }
GBR=GradientBoostingClassifier(random_state=SEED)

GBR_model=RandomizedSearchCV(GBR, param_distributions=GBR_prameter,cv=StratifiedKFold(n_splits=10),random_state=SEED)
predict_score(GBR_model,x_train_base,y_train_base,x_test_base,y_test,name)


c:\Users\sugii syuji\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\preprocessing\_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\sugii syuji\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\preprocessing\_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\sugii syuji\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\preprocessing\_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\sugii syuji\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklear

In [14]:
name="LR"
LR_prameter={
    'solver': ['newton-cg', 'liblinear'],
    'penalty': ['l2'],
    'C': [100, 10, 1.0, 0.1, 0.01],
    }
LR=LogisticRegression(random_state=SEED)

LR_model=RandomizedSearchCV(LR, param_distributions=LR_prameter,cv=StratifiedKFold(n_splits=10),random_state=SEED)
predict_score(LR_model,x_train_base,y_train_base,x_test_base,y_test,name)


c:\Users\sugii syuji\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\sugii syuji\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\optimize.py:319: ConvergenceWarning: newton-cg failed to converge at loss = 0.25382110637220845. Increase the number of iterations.
  warnings.warn(
c:\Users\sugii syuji\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\sugii syuji\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\optimize.py:319: ConvergenceWarning: newton-cg failed to conv

In [15]:
name="NB"
NB_model=GaussianNB()

predict_score(NB_model,x_train_base,y_train_base,x_test_base,y_test,name)


c:\Users\sugii syuji\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [16]:
name="KNN"
KNN_prameter={
    'n_neighbors': range(1, 21, 2),
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski']
    }
KNN=KNeighborsClassifier()

KNN_model=RandomizedSearchCV(KNN, param_distributions=KNN_prameter,cv=StratifiedKFold(n_splits=10),random_state=SEED)
predict_score(KNN_model,x_train_base,y_train_base,x_test_base,y_test,name)


c:\Users\sugii syuji\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\neighbors\_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\sugii syuji\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\neighbors\_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\sugii syuji\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\neighbors\_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\sugii syuji\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\neighbors\

In [17]:
name="MLP"
MLP_prameter={
    'hidden_layer_sizes': [(10,),(50,),(100,),(50, 50),(100, 50)],
    'alpha': [0.0001, 0.001, 0.01, 0.1, 1.0],
    'learning_rate_init': [0.001, 0.01, 0.1, 0.5, 1.0],
    'batch_size': [1, 32, 64, 128]
    }
MLP=MLPClassifier(random_state=SEED)

MLP_model=RandomizedSearchCV(MLP, param_distributions=MLP_prameter,cv=StratifiedKFold(n_splits=10),random_state=SEED)
predict_score(MLP_model,x_train_base,y_train_base,x_test_base,y_test,name)


c:\Users\sugii syuji\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1124: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\sugii syuji\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1124: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\sugii syuji\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1124: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\sugii syuji\AppData\Lo

In [18]:
print("f1",f1)
print("roc_auc",auc_score)

f1 {'DT': 0.13043478260869565, 'RF': 0.21978021978021978, 'GBR': 0.17857142857142858, 'LR': 0.09961685823754789, 'NB': 0.025261638397690366, 'KNN': 0.0989010989010989, 'MLP': 0.0871559633027523}
roc_auc {'DT': np.float64(0.6945237360489637), 'RF': np.float64(0.6354172024207512), 'GBR': np.float64(0.5682872156217319), 'LR': np.float64(0.6642000548612184), 'NB': np.float64(0.7778099230228531), 'KNN': np.float64(0.6144306434192254), 'MLP': np.float64(0.7344952767919902)}


In [19]:
for key in f1.keys():
    print(key,":",f1[key]," ",auc_score[key])

DT : 0.13043478260869565   0.6945237360489637
RF : 0.21978021978021978   0.6354172024207512
GBR : 0.17857142857142858   0.5682872156217319
LR : 0.09961685823754789   0.6642000548612184
NB : 0.025261638397690366   0.7778099230228531
KNN : 0.0989010989010989   0.6144306434192254
MLP : 0.0871559633027523   0.7344952767919902


In [1]:
import csv
data={
"DT" : [0.13043478260869565,0.6945237360489637],
"RF" : [0.21978021978021978,0.6354172024207512],
"GBR" : [0.17857142857142858,0.5682872156217319],
"LR" : [0.09961685823754789,0.6642000548612184],
"NB" : [0.025261638397690366,0.7778099230228531],
"KNN" : [0.0989010989010989,0.6144306434192254],
"MLP" : [0.0871559633027523,0.7344952767919902]
}

name="rb"
header=["","f1","roc_auc"]
with open("../result/"+name+".csv","w",newline="") as f:
    writer=csv.writer(f)
    writer.writerow(header)
    for key,value in data.items():
        writer.writerow([key]+[value[0]]+[value[1]])